In [1]:
# Implement batch gradient descent with early stopping for softmax regression without using scikit-Learn, only numpy.
# Use iris dataset


In [1]:
!python --version


Python 3.12.4


In [1]:
import numpy as np
from sklearn.datasets import load_iris
import pandas as pd

In [10]:
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

X = iris_df[["petal length (cm)", "petal width (cm)"]].values
y = iris.target

In [14]:
print(X[:5])

[[1.4 0.2]
 [1.4 0.2]
 [1.3 0.2]
 [1.5 0.2]
 [1.4 0.2]]


In [11]:
X_with_bias = np.c_[np.ones(len(X)), X]

In [15]:
print(X_with_bias[:5])

[[1.  1.4 0.2]
 [1.  1.4 0.2]
 [1.  1.3 0.2]
 [1.  1.5 0.2]
 [1.  1.4 0.2]]
